In [1]:
import os
os.chdir("..")

In [2]:
from app.pipelines import SalaryPredictionPipeline
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
gbr = GradientBoostingRegressor(n_estimators=200, max_depth=3, subsample=0.6424202471887338, learning_rate=0.03102740950912839)

In [4]:
pipeline = SalaryPredictionPipeline(gbr)
pipeline.build_pipeline()



AttributeError: 'GradientBoostingRegressor' object has no attribute 'estimators_'